In [ ]:
Rekognition didnt work that much

In [18]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def recognize(photo):
    #     photo='moviestars.jpg'
    client=boto3.client('rekognition')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    print('Detected faces for ' + photo)    
    for celebrity in response['CelebrityFaces']:
        print ('Name: ' + celebrity['Name'])
        print ('Id: ' + celebrity['Id'])
        print ('Position:')
        print ('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
        print ('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
        print ('Info')
        for url in celebrity['Urls']:
            print ('   ' + url)
        print()
        
        
        
        
        
        

def recognize_celebrities(photo):
    
    session = boto3.Session(profile_name='default')
    client = session.client('rekognition', region_name='us-east-1') #<<<<<<<<<<< era la ubicacion
    #     client = boto3.client('rekognition', region_name='ca-central-1')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    print('Detected faces for ' + photo)
    for celebrity in response['CelebrityFaces']:
        print('Name: ' + celebrity['Name'])
        print('Id: ' + celebrity['Id'])
        print('KnownGender: ' + celebrity['KnownGender']['Type'])
        print('Smile: ' + str(celebrity['Face']['Smile']['Value']))
        print('Position:')
        print('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
        print('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
        print('Info')
        for url in celebrity['Urls']:
            print('   ' + url)
        print()
    return len(response['CelebrityFaces'])

# def main():
#     photo = 'photo-name'
#     celeb_count = recognize_celebrities(photo)
#     print("Celebrities detected: " + str(celeb_count))

# if __name__ == "__main__":
#     main()
# for i in range(14,21):
#     photo = f'./test_images_movies/test{i}.png'
#     celeb_count = recognize_celebrities(photo)
#     print("Celebrities detected: " + str(celeb_count))

photo = './test_images_movies/test16.jpg'
celeb_count = recognize_celebrities(photo)
print("Celebrities detected: " + str(celeb_count))

Detected faces for ./test_images_movies/test16.jpg
Celebrities detected: 0


In [19]:
# Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

# rekognition = boto3.client('rekognition', region_name='ca-central-1')

# response = rekognition.search_faces_by_image(
#         CollectionId='famouspersons',
#         # CollectionId='famouspersons-images-lc',
#         Image={'Bytes':image_binary}                                       
#         )




import boto3

def get_celebrity_info(id):

    session = boto3.Session(profile_name='default')
    client = boto3.client('rekognition', region_name='us-east-1')

    # Display celebrity info
    print('Getting celebrity info for celebrity: ' + id)

    response = client.get_celebrity_info(Id=id)

    print(response['Name'])
    print('Further information (if available):')
    for url in response['Urls']:
        print(url)

def main():
    id = "celebrity-id"
    celebrity_info = get_celebrity_info(id)

if __name__ == "__main__":
    main()

Getting celebrity info for celebrity: celebrity-id


ClientError: An error occurred (ValidationException) when calling the GetCelebrityInfo operation: 1 validation error detected: Value 'celebrity-id' at 'id' failed to satisfy constraint: Member must satisfy regular expression pattern: [0-9A-Za-z]*

In [21]:
import boto3
import io
from PIL import Image

rekognition = boto3.client('rekognition', region_name='us-east-1')
dynamodb = boto3.client('dynamodb', region_name='us-east-1')

image_path = input("Enter path of the image to check: ")

image = Image.open(image_path)
stream = io.BytesIO()
image.save(stream,format="JPEG")
image_binary = stream.getvalue()


response = rekognition.search_faces_by_image(
        CollectionId='famouspersons',
        # CollectionId='famouspersons-images-lc',
        Image={'Bytes':image_binary}                                       
        )

found = False
for match in response['FaceMatches']:
    print (match['Face']['FaceId'],match['Face']['Confidence'])
    # try:        
    face = dynamodb.get_item(
        TableName='facerecognition',  
        Key={'RekognitionId': {'S': match['Face']['FaceId']}}
        )
    print ("Found Person: ",face)
     
    if 'Item' in face:
        print ("Found Person: ",face['Item']['FullName']['S'])
        found = True
    # except Exception as e:
    #     print("Error:",e)

if not found:
    print("Person cannot be recognized")

Enter path of the image to check: /Users/ivan/_AI/_TERM2/AML2404_AIML_lab/test_images_movies/test15.jpg


OSError: cannot write mode RGBA as JPEG